# **Alzheimer-Stall-Catchers-Point-Cloud**

This notebook contains necessary code for training data explained in **Point Cloud Based Approach** using 3D convolutional models such as:
- Resnet3D 18
- Resnet3D 101, 152, 200
- Densenet3D 121, 169, 201, 264

### Test GPU

This portion is to test available GPU on the machine. Torch models run on CUDA enabled devices, and it is recommended to use such a machine. In case of running the code on Google Colab, after connecting to a new session, be sure to test which GPU is provided for the session. **Tesla K80** is the slowest GPU that will take 5-6 times training time than **Tesla T4** or **Tesla P100**

In [ ]:
import torch
torch.cuda.get_device_name(0)

### Mount Google Drive 

**Skip this step if running on local machine**

Running the notebook on Colab requires data files and custom python modules copied from google drive or uploading the files to the colab session. However due to large dataset sizes, uploading data on each session if not a viable option. It is recommended to upload the data on a google drive, and running the following cell, allow file access to that drive for easily copying necessary files.

(Note that free google drive accounts give you only 15 GB of storage. In case data volume is bigger than that, you can create segments of the total data, hosting them on different drives, and then simply add shortcuts of the different drive folders to a single drive, giving you access to all the data from one google drive.)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Necessary Libaries

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import time
import shutil
import os
import glob

from sklearn.metrics import matthews_corrcoef as mcc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data
from torchvision.models.video import r3d_18

torch.manual_seed(100)

import csv

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Copy data

**Skip this cell if running on local machine**

After mounting google drive, you need to copy the necessary custom python modules, dataset and other files. The following cell first copies that large zipped data, and extracts them into the current colab workspace.

**traintestlist.zip** contains the train-validation split we had previously created for experimentation.

In [ ]:
!jar -xf "/content/drive/My Drive/Alzheimer/traintestlist.zip";

If you have not yet converted the original dataset to the point cloud dataset yet, head to the <a href="https://github.com/ClockWorkKid/Alzheimers-Stall-Catchers/tree/master/Dataset%20Visualization%20and%20Processing">**DATA VISUALIZATION AND PROCESSING**</a> section of the repository


For importing dataset from google drive, the data has been partitioned into 3 zip files, and each of the files are extracted into the colab session one after another.

In [ ]:
!jar -xf "/content/drive/My Drive/Alzheimer/micro_1.zip";
print("partition 1 imported")

!jar -xf "/content/drive/My Drive/Alzheimer/micro_2.zip";
print("partition 2 imported")

!jar -xf "/content/drive/My Drive/Alzheimer/micro_3.zip";
print("partition 3 imported")

Alzheimer Stall Catchers **Micro** dataset contains 2399 data samples, and there is a sanity check to see whether all the data files have been imported successfully. If running on local machine, you can just specify the data folder to check if all files are there.

In [ ]:
files = [f for f in glob.glob("micro/" + "*" + ".pt", recursive=True)]
print("Total: " + str(len(files)) + " should be 2399")

You need to import custom modules for Resnet and Densenet 3D models. In case of local machine, simply copy these files to the notebook directory and head on to the next steps. In case of using Colab, copy the files from drive.

In [ ]:
shutil.copyfile("/content/drive/My Drive/Alzheimer/resnet.py", "resnet.py"")
shutil.copyfile("/content/drive/My Drive/Alzheimer/densenet.py", "densenet.py")

If you need to remove any folders from the colab workspace fully including any subdirectories and files, simply run the command:
```
! rm -rf <folder_name>
```

For example, running the following code removes the traintestlist folder from the session.
```
! rm -rf traintestlist
```


# Dataset

Torch deep learning models use dataloaders while importing data during training/testing passing them to CPU/GPU. Torch datasets can be easily modified to suit custom datasets.

### Balanced Batch Sampler

Balanced batch sampler can be used to create training batches from heavily imbalanced datasets (most biomedical datasets). The process is to oversample the data class with the lower instance count, so that each batch has 50:50 ratio while training. In each epoch, the model might see the same data from the lower count data class.

In [ ]:
import torchvision
import torch.utils.data
import random


class BalancedBatchSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
        self.dataset = {}
        self.balanced_max = 0
        # Save all the indices for all the classes
        for idx in range(0, len(dataset)):
            label = self._get_label(dataset, idx)
            if label not in self.dataset:
                self.dataset[label] = []
            self.dataset[label].append(idx)
            self.balanced_max = len(self.dataset[label]) \
                if len(self.dataset[label]) > self.balanced_max else self.balanced_max
        
        # Oversample the classes with fewer elements than the max
        for label in self.dataset:
            while len(self.dataset[label]) < self.balanced_max:
                self.dataset[label].append(random.choice(self.dataset[label]))
    
        self.keys = list(self.dataset.keys())
        self.currentkey = 0

    def __iter__(self):
        while len(self.dataset[self.keys[self.currentkey]]) > 0:
            yield self.dataset[self.keys[self.currentkey]].pop()
            self.currentkey = (self.currentkey + 1) % len(self.keys)

    
    def _get_label(self, dataset, idx):
        dataset_type = type(dataset)
        if dataset_type is torchvision.datasets.MNIST:
            return dataset.train_labels[idx].item()
        elif dataset_type is torchvision.datasets.ImageFolder:
            return dataset.imgs[idx][1]
        else:
            (image_sequence, target) = dataset.__getitem__(idx)
            return target

    def __len__(self):
        return self.balanced_max*len(self.keys)

### Custom Dataset Class

The dataloader here loads point cloud dataset saved in disk in .pt file format (torch tensor data).

Here is a link from <a href="https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel">stanford.edu</a> that beautifully explains how to create your own custom dataloader for torch.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class VoxelDataset(Dataset):
    def __init__(self, dataset_path, split_path, split_number, training):
        self.training = training
        self.sequences, self.labels = self._extract_sequence_paths_and_labels(dataset_path, split_path, split_number,
                                                                              training)  # creating a list of directories where the extracted frames are saved
        self.label_names = ["Non-stalled", "Stalled"]  # Getting the label names or name of the class
        self.num_classes = len(self.label_names)  # Getting the number of class


    def _extract_sequence_paths_and_labels(
            self, dataset_path, split_path="data/traintestlist", split_number=0, training=True
    ):
        """ Extracts paths to sequences given the specified train / test split """
        fn = f"fold_{split_number}_train.csv" if training else f"fold_{split_number}_test.csv"
        split_path = os.path.join(split_path, fn)
        df = pd.read_csv(split_path)
        file_name = df['filename'].values
        all_labels = df['class'].values
        sequence_paths = []
        classes = []
        for i, video_name in enumerate(file_name):
            seq_name = video_name.split(".mp4")[0]
            sequence_paths += [os.path.join(dataset_path, seq_name).replace('\\', '/')]
            classes += [all_labels[i]]
        return sequence_paths, classes

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence_path = self.sequences[index % len(self)]
        target = self.labels[index % len(self)]

        X = torch.load(sequence_path + ".pt")

        return X, target


### Augmentation

Torch dataloaders usually include methods for data augmentation. In those methods, each data instance is given a random augmentation while creating a new data batch, i.e. each data point is slightly modified during each epoch. However in our approach, we wanted to feed all the possible augmentations in a single epoch, necessarily creating 16 copies of each data point and giving them a specific augmentation such as rotation along Z-axis, upside down rotation, mirroring and combination of these. From our tests despite training time linearly increasing with augmentation volume, it gave huge performance boost because of reduced overfitting in all of the training models.

In [ ]:
def augment(images, labels, augment_volume=2):

  [instances, channel, depth, height, width] = images.shape

  images_aug = torch.zeros((instances*augment_volume, channel, depth, height, width))
  labels_aug = torch.zeros(instances*augment_volume)

  for aug_type in range(augment_volume):
    augmented = im_aug(images, aug_type)
    
    images_aug[aug_type*instances:(aug_type+1)*instances, :, :, :, :] = augmented
    labels_aug[aug_type*instances:(aug_type+1)*instances] = labels

  return images_aug.float(), labels_aug.long()



def im_aug(images, aug_type):

    if aug_type == 0:
      return images

    elif aug_type == 1:
      return torch.rot90(images, 1, [3,4])

    elif aug_type == 2:
      return torch.rot90(images, 2, [3,4])

    elif aug_type == 3:
      return torch.rot90(images, 3, [3,4])

    elif aug_type == 4:
      temp = torch.flip(images, [2,3])
      return temp

    elif aug_type == 5:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 6:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 7:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 3, [3,4])

    elif aug_type == 8:
      temp = torch.transpose(images, 3, 4)
      return temp

    elif aug_type == 9:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 10:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 11:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 3, [3,4])

    elif aug_type == 12:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return temp

    elif aug_type == 13:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 14:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 15:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 3, [3,4])

# Visualization

Confusion matrix is created for each epoch and saved to a folder

In [ ]:
def conf_mat(cf, epoch, acc, mcc, y_true, train_loss, train_acc, test_loss, test_acc):
  try:
    plt.imshow(cf,cmap=plt.cm.RdYlGn,interpolation='nearest')
    plt.colorbar()
    plt.title('Confusion Matrix without Normalization')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    tick_marks = np.arange(len(set(y_true))) # length of classes
    class_labels = ['Non Stalled','Stalled']
    tick_marks
    plt.xticks(tick_marks,class_labels)
    plt.yticks(tick_marks,class_labels)
    # plotting text value inside cells
    thresh = cf.max() / 2.
    for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
        plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] >thresh else 'black')
    #plt.show()
    os.makedirs("confusion_matrix", exist_ok=True)
    plt.savefig(f"confusion_matrix/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
    plt.close('all')

    os.makedirs("loss_acc_curve", exist_ok=True)

    ##creating subplot for loss,acc
    fig1, axs = plt.subplots(4,sharex=True,constrained_layout=True)
    axs[0].plot(train_loss, color = "red") 
    axs[0].set_title("Train loss")
    axs[1].plot(train_acc, color = "blue") 
    axs[1].set_title("Train Accuracy")
    axs[2].plot(test_loss, color = "green")
    axs[2].set_title("Test Loss")
    axs[3].plot(test_acc) 
    axs[3].set_title("Test Accuracy")
    #fig1.tight_layout()
    #plt.show()
    fig1.savefig(f"loss_acc_curve/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
    plt.close(fig1)
  except:
    print("MCC is 0")

# Training/Validation Batch Creation using Dataloader

In [ ]:
batch_size = 32

split_number = 3

augmentation_enabled = True     # Set this flag to False if augmentation is not desired
augment_volume = 16             # How many times data will be augmented (min 1 max 16)

# If augmentation is enabled, original batch size is reduced, otherwise the
# data will not fit in the GPU memory
if augmentation_enabled ==  True:
  batch_size = int(batch_size/augment_volume)


dataset_path = 'micro'          # Path to the dataset directory
split_path = 'traintestlist'    # Path to the folder containing train test split csv files

depth, height, width = 32, 64, 64   # Input data dimension


# Define training set
train_dataset_vox = VoxelDataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    training=True,
)
# Not using balanced batch sampler, shuffle enabled
train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,shuffle=True, num_workers=4)
# Using balanced batch sampler
# train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,sampler=BalancedBatchSampler(train_dataset_vox),shuffle=False, num_workers=4)


# Define validation set
test_dataset_vox = VoxelDataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    training=False,
)
test_dataloader_vox = DataLoader(test_dataset_vox, batch_size=batch_size, shuffle=False, num_workers=4)


# **Network Model**

As mentioned at the beginning of the notebook, Resnet3D/Densenet3D models have been used for training. The model must be imported and sent to device prior to training, and depending on which model you wish to train, you have to import that model either from torch libary (resnet3D 18) or our custorm python modules (resnet.py/densenet.py)

###**Resnet3D 18**
```
from torchvision.models.video import r3d_18

model = r3d_18(pretrained = False) # Change to true for a pretrained model 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.fc.out_features = 2
```

###**Resnet 101, 152, 200**
```
import resnet

# models are 101 152 200
model = resnet.resnet101(   
                num_classes=2,
                shortcut_type='B',
                sample_size=64,
                sample_duration=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
```

###**Densenet 121, 169, 201, 264**
```
from densenet import generate_model

# models are 121 169 201 264
model = generate_model(model_depth = 264 , num_classes = 2) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
```
###**Loading Model Checkpoints**
Finally, if you are continuing the training form a previous model checkpoint/ some other pretrained weight files, be sure to load them into the model.
```
checkpoint_model = "weight_3D.pth"  # weight file location

model.load_state_dict(torch.load(checkpoint_model))
```

In [ ]:
from densenet import generate_model

model = generate_model(model_depth = 264 , num_classes = 2) # values are 121 169 201 264
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [ ]:
shutil.copyfile("/content/drive/My Drive/Alzheimer/densenet169_ep_11_acc_89.316_mcc_0.735.pth", "weight_3D.pth")
checkpoint_model = "weight_3D.pth"

model.load_state_dict(torch.load(checkpoint_model))

# Loss Function

### Class Balance Loss Function

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def CB_loss(labels, logits, samples_per_cls, no_of_classes, loss_type, beta, gamma):
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes

    labels_one_hot = F.one_hot(labels, no_of_classes).float()

    weights = torch.tensor(weights).float().to(device)
    weights = weights.unsqueeze(0)
    weights = weights.repeat(labels_one_hot.shape[0],1) * labels_one_hot
    weights = weights.sum(1)
    weights = weights.unsqueeze(1)
    weights = weights.repeat(1,no_of_classes)

    if loss_type == "focal":
        cb_loss = focal_loss(labels_one_hot, logits, weights, gamma)
    elif loss_type == "sigmoid":
        cb_loss = F.binary_cross_entropy_with_logits(input = logits,target = labels_one_hot, weights = weights)
    elif loss_type == "softmax":
        pred = logits.softmax(dim = 1)
        cb_loss = F.binary_cross_entropy(input = pred, target = labels_one_hot, weight = weights)
    return cb_loss

### Class Balance Loss Hyperparameters

In [ ]:
no_of_classes = 2
beta = 0.9999
gamma = 2.0
samples_per_cls = [int(batch_size*0.7), batch_size - int(batch_size*0.7)]   #class sample number in one mini batch(For example we have 70:30 ratio data of mini batch_size of 50)
loss_type = "softmax"

# Training Hyperparameters

While training the models for 9-10 epochs, the validation MCC score tends to start growing again instead of convering. At that point, it is generally a good idea to stop training and increasing the **L2 regularization** value from 1e-4 to somewhere around 5e-4 or 8e-4. This helps in reducing overfitting the model. Again this might result in the model having a high training bias, which can be further improved by loweing the **learning rate** to 5e-4 or 2e-4

In [ ]:
num_epochs = 50


learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Validation

This function computes the validation set accuracy and MCC. If you are running on colab instead of a local machine, the following line can be modified to save the weight files from each epoch on google drive instead of the colab runtime, which saves you from losing training progress due to runtime disconnection.

```
torch.save(model.state_dict(), f"model_checkpoints/ep_{epoch}_acc_{round(accuracy.item(),3)}_mcc_{round(mcc_score.item(),3)}.pth")
```


In [ ]:
def validation(epoch,train_loss, train_acc, test_loss, test_acc):

  correct = 0
  total = 0
  t_loss = 0
  y_true = []
  y_pred = []
  

  model.eval()

  for images, labels in test_dataloader_vox:

      y_true = np.append(y_true, labels.numpy())

      images = images.view(-1,3,depth,height,width)
      
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      with torch.no_grad():
      
        # Forward propagation
        outputs = model(test)
        t_loss += CB_loss(labels, outputs, samples_per_cls, no_of_classes,loss_type, beta, gamma)

        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())
        # Total number of labels
        total += len(labels)
        correct += (predicted == labels).sum()

  model.train()
  loss = t_loss.cpu().numpy() / float(total)
  test_loss.append(loss)
  accuracy = 100 * correct / float(total)
  test_acc.append(accuracy)

  print('TESTING ---> Epoch: {} Loss: {} Accuracy: {} %'.format(epoch, round(loss,3), round(accuracy.item(),3)))
  mcc_score = mcc(y_true, y_pred)
  print(f"Validation MCC {round(mcc_score,4)}")


  global MCC_SCORE
  if MCC_SCORE < mcc_score:
      MCC_SCORE = mcc_score
  os.makedirs("model_checkpoints", exist_ok=True)
  try:
    torch.save(model.state_dict(), f"model_checkpoints/ep_{epoch}_acc_{round(accuracy.item(),3)}_mcc_{round(mcc_score.item(),3)}.pth")
  except:
    pass
  cf =confusion_matrix(y_true, y_pred)
  #print(cf)
  conf_mat(cf, epoch, accuracy, mcc_score, y_true, train_loss, train_acc, test_loss, test_acc)

  return test_loss, test_acc

# **Training**

In [ ]:
### TRAINING code

MCC_SCORE = -1


train_loss = []   #to keep track of loss with respect to number of epoch 
test_loss = []
iteration_list = []
train_acc = []
test_acc = []

for epoch in tqdm(range(num_epochs)):
    #count = 1
    accuracy_list = []
    loss_list = []
    for i, (images, labels) in enumerate(train_dataloader_vox):
        correct = 0
        
        if augmentation_enabled == True:
          images, labels = augment(images, labels, augment_volume)

        images = images.view(-1,3,depth,height,width)


        train_img = Variable(images.to(device), requires_grad=True)
        labels = Variable(labels.to(device), requires_grad=False)

        # Clear gradients
        optimizer.zero_grad()
        # Forward propagation
        outputs = model(train_img)
        # Calculate softmax and ross entropy loss
        loss = CB_loss(labels, outputs, samples_per_cls, no_of_classes,loss_type, beta, gamma)

        
        # Calculating gradients
        loss.backward()
        # Update parameters
        optimizer.step()
        
        predicted = torch.max(outputs.data, 1)[1]   
        correct = (predicted == labels).sum()
        total = len(labels)        
        accuracy = 100 * correct / float(total)

        accuracy_list = np.append(accuracy_list, accuracy.cpu().numpy())
        loss_list = np.append(loss_list, loss.detach().cpu().numpy())


    final_loss = np.mean(loss_list)
    final_acc = np.mean(accuracy_list)
    print(" ")
    print(f"TRAINING ---> Epoch: {epoch} Loss: {round(final_loss,4)} Accuracy: {round(final_acc,4)}")
    train_loss.append(final_loss)
    train_acc.append(final_acc)
    gc.collect() 
    test_loss , test_acc = validation(epoch, train_loss, train_acc, test_loss, test_acc)
    
    train_dataset_vox = VoxelDataset(
        dataset_path=dataset_path,
        split_path=split_path,
        split_number=split_number,
        training=True,
    )
    train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,shuffle=True, num_workers=4)
    #train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,sampler=BalancedBatchSampler(train_dataset_vox),shuffle=False, num_workers=4)


**TIP:** This training could take several hours depending on how many epochs you chose. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```